In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 983 100 983 0 0 32766 0 --:--:-- --:--:-- --:--:-- 32766
Warning: apt-key output should not be parsed (stdout is not a terminal)
OK
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 79 100 79 0 0 1680 0 --:--:-- --:--:-- --:--:-- 1680
Hit:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Ign:2 http://repos.azulsystems.com/ubuntu stable InRelease
Hit:3 http://repos.azulsystems.com/ubuntu stable Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://repos.azul.com/zulu/deb stable InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
msodbcsql17 is already the newest version (17.8.1.1-1).
The following packages were automatically installed and are no longer required:
 libcap2-bin libpam-cap zulu-8 zulu-repo
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.

In [ ]:
import os
import json

from concurrent import futures
import time

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql import SQLContext

from edm.batch_ingestion.param_config_parser import ParamConfigParser
from edm.batch_ingestion.schema_evolution.schema_evolution import SchemaEvolution
from edm.batch_ingestion.landing_to_raw.landing_to_raw_processor import LandingToRawProcessor
from edm.utils.keyvault_utility import KeyvaultSecretsUtility

In [ ]:
dbutils.widgets.text('source_file_metadata', '')
dbutils.widgets.text('spn_id', '')
dbutils.widgets.text('tenant_id', '')
dbutils.widgets.text('kv_name', '')
dbutils.widgets.text('adls_account_name', '')
dbutils.widgets.text('config_path', '')

In [ ]:
source_file_metadata = json.loads(dbutils.widgets.get('source_file_metadata').replace('\\', ""))
spn_id = dbutils.widgets.get('spn_id')
tenant_id = dbutils.widgets.get('tenant_id')
kv_name = dbutils.widgets.get('kv_name')
adls_account_name = dbutils.widgets.get('adls_account_name')
config_path = dbutils.widgets.get('config_path')

In [ ]:
spn_password = os.getenv('SPN_PASSWORD', None)

In [ ]:
spn_credentials = dict()
spn_credentials['spn_id'] = spn_id
spn_credentials['spn_password'] = spn_password
spn_credentials['tenant_id'] = tenant_id

In [ ]:
# Leverage distributive processing
def run_landing_obj(item):
    print(item)
    spark = SparkSession.builder.appName('').getOrCreate()
    dbutils = DBUtils(spark)
    sqlContext = SQLContext(spark)
    parser_obj = ParamConfigParser(
        item['Source'], item['Country'], item['FileName'], item['PipelineLogId'],
        spark, dbutils, spn_credentials=spn_credentials,
        kv_name=kv_name, adls_account_name=adls_account_name,
        config_path=config_path
    )
    object_name = parser_obj.run()
    if(object_name is not None):
        # Performing schema evolution and polybase table refresh
        schema_evol_objs = SchemaEvolution(
            item['Source'], item['Country'], item['FileName'], 
            object_name, item['PipelineLogId'],
            spark, dbutils, spn_credentials=spn_credentials,
            kv_name=kv_name, adls_account_name=adls_account_name,
            config_path=config_path
        )
        schema_evol_objs.run()

        # Landing to Raw
        landing_obj = LandingToRawProcessor(
                item['Source'], item['Country'], item['FileName'], object_name, item['PipelineLogId'],
                spark, sqlContext, dbutils, spn_credentials=spn_credentials,
                kv_name=kv_name, adls_account_name=adls_account_name,
                config_path=config_path
            )
        landing_obj.run()

In [ ]:
t1 = time.perf_counter()
with futures.ThreadPoolExecutor() as executor:
  result = executor.map(run_landing_obj,source_file_metadata)
  print(result)

t2 = time.perf_counter()
t = t2 - t1
print(f"total {t} secs taken")

{'PipelineLogId': '14123', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_acctlmt_20200908_1000.csv'}
3289829 | 2021-08-11 12:41:48,023 | INFO | ThreadPoolExecutor-14_0 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
3289830 | 2021-08-11 12:41:48,023 | INFO | ThreadPoolExecutor-14_0 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation
{'PipelineLogId': '14124', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_accinfo_20200908_1000.csv'}
3289833 | 2021-08-11 12:41:48,027 | INFO | ThreadPoolExecutor-14_1 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
3289834 | 2021-08-11 12:41:48,027 | INFO | ThreadPoolExecutor-14_1 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation
{'PipelineLogId': '14125', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_accinfo_20200907_1000.csv'}
{'PipelineLogId': '14126', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_acctlmt_20200907_1000.csv'}3289836 | 2021-08-11 12:41:48,029 | INFO | ThreadPoolExecutor-14_2 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
{'PipelineLogId': '14127', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_acctlmt_20200906_1000.csv'}3289839 | 2021-08-11 12:41:48,032 | INFO | ThreadPoolExecutor-14_2 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation


3289841 | 2021-08-11 12:41:48,035 | INFO | ThreadPoolExecutor-14_3 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
{'PipelineLogId': '14128', 'Source': 'ebbs', 'Country': 'ph', 'FilePath': 'incoming/data/ebbs/ph', 'FileName': 'ebbs_ph_accinfo_20200906_1000.csv'}<generator object Executor.map.<locals>.result_iterator at 0x7f8a749724d0>

3289842 | 2021-08-11 12:41:48,035 | INFO | ThreadPoolExecutor-14_4 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
3289842 | 2021-08-11 12:41:48,036 | INFO | ThreadPoolExecutor-14_3 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation
3289843 | 2021-08-11 12:41:48,036 | INFO | ThreadPoolExecutor-14_5 | edm.batch_ingestion.param_config_parser: 602 run(): Starting config Param Parser
3289845 | 2021-08-11 12:41:48,038 | INFO | ThreadPoolExecutor-14_4 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation
3289854 | 2021-08-11 12:41:48,047 | INFO | ThreadPoolExecutor-14_5 | edm.batch_ingestion.param_config_parser: 603 run(): Started Initialisation
3290648 | 2021-08-11 12:41:48,842 | INFO | ThreadPoolExecutor-14_5 | edm.batch_ingestion.param_config_parser: 608 run(): Ended Initialisation
3290649 | 2021-08-11 12:41:48,843 | INFO | ThreadPoolExecutor-14_5 | edm.batch_ingestion.param_config_parser: 609 run(): Starting Parsing XML Files
3290749 | 2021-08-11 12:41:48,943 | INFO | ThreadPoolExecutor-14_2 | edm.batch_ingestion.param_config_parser: 608 run(): Ended Initialisation
3290751 | 2021-08-11 12:41:48,944 | INFO | ThreadPoolExecutor-14_2 | edm.batch_ingestion.param_config_parser: 609 run(): Starting Parsing XML Files
3290757 | 2021-08-11 12:41:48,950 | INFO | ThreadPoolExecutor-14_0 | edm.batch_ingestion.param_config_parser: 608 run(): Ended Initialisation
3290758 | 2021-08-11 12:41:48,951 | INFO | ThreadPoolExecutor-14_0 | edm.batch_ingestion.param_config_parser: 609 run(): Starting Parsing XML Files
3290762 | 2021-08-11 12:41:48,955 | INFO | ThreadPoolExecutor-14_1 | edm.batch_ingestion.param_config_parser: 608 run(): Ended Initialisation
3290763 | 2021-08-11 12:41:48,956 | INFO | ThreadPoolExecutor-14_1 | edm.batch_ingestion.param_config_parser: 609 run(): Starting Parsing XML Files
3290816 | 2021-08-11 12:41:49,010 | INFO | ThreadPoolExecutor-14_3 | edm.batch_ingestion.param_config_parser: 608